In [ ]:
from dataclasses import dataclass

@dataclass
class Config:
    interactions_path: str = "data/cb_baseline.parquet"
    items_path: str = "data/geo_feature_matrix.csv"
    artifacts_dir: str = "artifacts"

    embed_dim: int = 128
    hidden_dim: int =512
    lr: float = 1e-3
    weight_decay: float = 1e-5
    epochs: int = 30 #10
    batch_size: int = 2048 #2048

    # label rule
    # centered_rating > 0 => positive
    pos_threshold: float = 0.0

    # evaluation    
    k: int = 20
    test_size: float = 0.2
    random_state: int = 42

In [42]:
import torch
import torch.nn as nn

class TwoTower(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int = 128, embed_dim: int = 64):
        super().__init__()
        self.item_tower = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, embed_dim),
        )
        self.user_tower = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, embed_dim),
        )

    def forward(self, user_x, item_x):
        u = self.user_tower(user_x)
        i = self.item_tower(item_x)
        # dot product
        return (u * i).sum(dim=1)

In [43]:
import os
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

def load_raw(interactions_path: str, items_path: str):
    interactions = pd.read_parquet(interactions_path)
    items = pd.read_csv(items_path)
    def load_raw(interactions_path: str, items_path: str):
        interactions = pd.read_parquet(interactions_path)
        items = pd.read_csv(items_path)

        # ✅ 关键：清理列名空格/不可见字符，避免 merge / 筛列失败
        interactions.columns = interactions.columns.str.strip()
        items.columns = items.columns.str.strip()

        # ✅ 快速自检：items到底有多少列
        print("[DEBUG] items shape:", items.shape)
        print("[DEBUG] first 30 item columns:", list(items.columns[:30]))
        print("[DEBUG] last  30 item columns:", list(items.columns[-30:]))

        interactions["business_id"] = interactions["business_id"].astype(str)
        items["business_id"] = items["business_id"].astype(str)
        interactions["user_id"] = interactions["user_id"].astype(str)

        return interactions, items


    # unify types
    interactions["business_id"] = interactions["business_id"].astype(str)
    items["business_id"] = items["business_id"].astype(str)
    interactions["user_id"] = interactions["user_id"].astype(str)

    return interactions, items

def get_feature_cols(items: pd.DataFrame):
    # 先排除 id
    candidate = items.drop(columns=["business_id"], errors="ignore")

    # 只保留“看起来像数值”的列：要么本来就是数值 dtype，
    # 要么能被安全转换成数值（比如object但内容是 "25.0"）
    numeric_cols = []
    for c in candidate.columns:
        s = pd.to_numeric(candidate[c], errors="coerce")
        # 至少有一部分不是 NaN 才算有效特征
        if s.notna().mean() > 0.5:
            numeric_cols.append(c)

    print("[DEBUG] feature cols:", len(numeric_cols))
    return numeric_cols


def make_merged_df(interactions: pd.DataFrame, items: pd.DataFrame):
    df = interactions.merge(items, on="business_id", how="inner")
    return df

def make_label(df: pd.DataFrame, pos_threshold: float = 0.0):
    if "centered_rating" not in df.columns:
        raise ValueError("cb_baseline.parquet must contain centered_rating column.")
    df["label"] = (df["centered_rating"] > pos_threshold).astype(int)
    return df

def numeric_log1p_clip(df: pd.DataFrame, cols):
    x = df[cols].apply(pd.to_numeric, errors="coerce").fillna(0.0)
    x = x.clip(lower=0.0)
    return np.log1p(x.to_numpy(dtype=np.float32))


def build_user_profiles(df: pd.DataFrame, feature_cols):
    # user profile = mean of positive items' content features
    pos = df[df["label"] == 1].copy()
    if len(pos) == 0:
        raise ValueError("No positive samples found. Check pos_threshold or centered_rating distribution.")
    for c in feature_cols:
        pos[c] = pd.to_numeric(pos[c], errors="coerce")
    pos[feature_cols] = pos[feature_cols].fillna(0.0)
    profiles = pos.groupby("user_id")[feature_cols].mean()
    return profiles

def build_training_matrix(df: pd.DataFrame, user_profiles: pd.DataFrame, feature_cols):
    # attach user_profile columns to each interaction row
    train_df = df.merge(user_profiles, on="user_id", how="inner", suffixes=("", "_user"))
    if len(train_df) == 0:
        raise ValueError("train_df is empty after merging user_profiles. Possibly no users with positives in df.")
    user_feature_cols = [f"{c}_user" for c in feature_cols]
    return train_df, user_feature_cols

def fit_scaler_and_transform(train_df: pd.DataFrame, feature_cols, user_feature_cols):
    X_item = numeric_log1p_clip(train_df, feature_cols)
    X_user = numeric_log1p_clip(train_df, user_feature_cols)
    y = train_df["label"].to_numpy(dtype=np.float32)

    scaler = StandardScaler()
    X_item = scaler.fit_transform(X_item).astype(np.float32)
    X_user = scaler.transform(X_user).astype(np.float32)
    return X_user, X_item, y, scaler

def transform_all_items(items: pd.DataFrame, feature_cols, scaler: StandardScaler):
    # IMPORTANT: recommend over UNIQUE restaurants, not interaction rows
    tmp = items[["business_id"] + feature_cols].copy()
    X_item = numeric_log1p_clip(tmp, feature_cols)
    X_item = scaler.transform(X_item).astype(np.float32)
    item_ids = tmp["business_id"].to_numpy()
    return item_ids, X_item

def save_metadata(artifacts_dir: str, feature_cols):
    os.makedirs(artifacts_dir, exist_ok=True)
    with open(os.path.join(artifacts_dir, "feature_cols.json"), "w") as f:
        json.dump(feature_cols, f)

In [44]:
import torch, sys
print("python:", sys.version)
print("torch:", torch.__version__)
print("torch cuda:", torch.version.cuda)
print("cuda available:", torch.cuda.is_available())
print("device count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("gpu:", torch.cuda.get_device_name(0))
    print("archs:", torch.cuda.get_arch_list())


python: 3.12.12 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 20:05:38) [MSC v.1929 64 bit (AMD64)]
torch: 2.10.0+cu128
torch cuda: 12.8
cuda available: True
device count: 1
gpu: NVIDIA GeForce RTX 5070
archs: ['sm_70', 'sm_75', 'sm_80', 'sm_86', 'sm_90', 'sm_100', 'sm_120']


In [45]:
import os
import joblib
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import random
from tensorboardX import SummaryWriter
import time




class BPRDataset(Dataset):
    def __init__(self, df_pos, user_cache, feature_cols, items_df, scaler):
        self.df_pos = df_pos[["user_id","business_id"]].reset_index(drop=True)
        self.user_cache = user_cache
        self.feature_cols = feature_cols

        # 预处理好的“全量item特征矩阵”
        self.item_ids, self.item_X = transform_all_items(items_df, feature_cols, scaler)
        self.id2idx = {bid:i for i,bid in enumerate(self.item_ids)}
        self.all_item_ids = list(self.id2idx.keys())

        self.user_pos = (
            df_pos.groupby("user_id")["business_id"].apply(set).to_dict()
        )

    def __len__(self):
        return len(self.df_pos)

    def __getitem__(self, idx):
        uid = self.df_pos.loc[idx, "user_id"]
        pos_bid = self.df_pos.loc[idx, "business_id"]

        # user vector：直接缓存取
        u_x = self.user_cache[uid].astype(np.float32)

        # pos item
        pos_i = self.item_X[self.id2idx[pos_bid]]

        # sample neg item
        while True:
            neg_bid = random.choice(self.all_item_ids)
            if neg_bid not in self.user_pos.get(uid, set()):
                break
        neg_i = self.item_X[self.id2idx[neg_bid]]

        return torch.from_numpy(u_x), torch.from_numpy(pos_i), torch.from_numpy(neg_i)


class RecDataset(Dataset):
    def __init__(self, X_user, X_item, y):
        self.X_user = torch.from_numpy(X_user)
        self.X_item = torch.from_numpy(X_item)
        self.y = torch.from_numpy(y)
    def __len__(self):
        return self.y.shape[0]
    def __getitem__(self, idx):
        return self.X_user[idx], self.X_item[idx], self.y[idx]

def bpr_loss(pos_score, neg_score):
    return -torch.log(torch.sigmoid(pos_score - neg_score) + 1e-8).mean()



def main():

    cfg = Config()
    os.makedirs(cfg.artifacts_dir, exist_ok=True)

    interactions, items = load_raw(cfg.interactions_path, cfg.items_path)
    feature_cols = get_feature_cols(items)

    df = make_merged_df(interactions, items)
    df = make_label(df, cfg.pos_threshold)
    df["review_date"] = pd.to_datetime(df["review_date"], errors="coerce")
    df = df.dropna(subset=["review_date"])

    cutoff = df["review_date"].quantile(0.8)
    df_train = df[df["review_date"] <= cutoff].copy()
    df_test  = df[df["review_date"] >  cutoff].copy()

    # ---- time split ----
    df["review_date"] = pd.to_datetime(df["review_date"], errors="coerce")
    df = df.dropna(subset=["review_date"])

    cutoff = df["review_date"].quantile(0.8)  # 前80%做train，后20%做test
    df_train = df[df["review_date"] <= cutoff].copy()
    df_test  = df[df["review_date"] >  cutoff].copy()

    # 用户在 train 中见过的 items（需要从推荐列表里排除）
    seen = (
        df_train.groupby("user_id")["business_id"]
        .apply(set)
        .to_dict()
    )
    all_item_ids = items["business_id"].astype(str).unique().tolist()

    # 用户在 train 的正样本集合（采负样本时避免采到正样本）
    user_pos = (
        df_train[df_train["label"] == 1]
        .groupby("user_id")["business_id"]
        .apply(set)
        .to_dict()
    )

    # ground truth 用 test 的正样本（更合理）
    pos_test = df_test[df_test["label"] == 1][["user_id", "business_id"]].copy()
    gt = pos_test.groupby("user_id")["business_id"].apply(list).to_dict()

    print("cutoff:", cutoff)
    print("train rows:", len(df_train), "test rows:", len(df_test))

    # keep only feature columns that exist after merge
    feature_cols = [c for c in feature_cols if c in df.columns]
    if len(feature_cols) == 0:
        raise ValueError("No feature columns found in merged df. Check items csv columns.")

    user_profiles = build_user_profiles(df_train, feature_cols)
    train_df, user_feature_cols = build_training_matrix(df_train, user_profiles, feature_cols)

    X_user, X_item, y, scaler = fit_scaler_and_transform(train_df, feature_cols, user_feature_cols)
    user_cache = {}
    U = user_profiles[feature_cols].to_numpy(dtype=np.float32)  # (num_users, d)
    U_df = pd.DataFrame(U, columns=feature_cols)
    U_df = numeric_log1p_clip(U_df, feature_cols)
    U_scaled = scaler.transform(U_df).astype(np.float32)

    for idx, uid in enumerate(user_profiles.index):
        user_cache[uid] = U_scaled[idx]
    X_user_tr, X_user_te, X_item_tr, X_item_te, y_tr, y_te = train_test_split(
        X_user, X_item, y,
        test_size=cfg.test_size,
        random_state=cfg.random_state,
        stratify=y if len(np.unique(y)) > 1 else None
    )

    train_loader = DataLoader(RecDataset(X_user_tr, X_item_tr, y_tr), batch_size=cfg.batch_size, shuffle=True)
    test_loader  = DataLoader(RecDataset(X_user_te, X_item_te, y_te), batch_size=cfg.batch_size*2, shuffle=False)
    # 只用 train 的正样本做 BPR 训练
    df_pos_train = df_train[df_train["label"] == 1][["user_id", "business_id"]].copy()

    train_loader = DataLoader(
        BPRDataset(df_pos_train, user_cache, feature_cols, items, scaler),
        batch_size=cfg.batch_size,
        shuffle=True,
        num_workers=0,
        pin_memory=True,
    )


    # test 仍然可以用 RecDataset 做 AUC（可选）
    test_loader = DataLoader(
        RecDataset(X_user_te, X_item_te, y_te),
        batch_size=cfg.batch_size*2,
        shuffle=False
    )

    if torch.cuda.is_available():
        device = torch.device("cuda")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")
    print("device:", device)
    model = TwoTower(input_dim=len(feature_cols), hidden_dim=cfg.hidden_dim, embed_dim=cfg.embed_dim).to(device)
    opt = torch.optim.Adam(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    loss_fn = nn.BCEWithLogitsLoss()
    print(torch.backends.mps.is_available(), torch.backends.mps.is_built())
    print("device:", device)
    print("train samples:", len(X_user_tr), "test samples:", len(X_user_te))
    print("num features:", len(feature_cols))

    run_id = time.strftime("%Y%m%d-%H%M%S")
    log_dir = os.path.join("runs", f"twotower_{run_id}")
    writer = SummaryWriter(log_dir=log_dir)

    writer.add_text(
        "hparams",
        f"embed_dim={cfg.embed_dim}, hidden_dim={cfg.hidden_dim}, lr={cfg.lr}, "
        f"wd={cfg.weight_decay}, bs={cfg.batch_size}"
    )

    print("TensorBoard log_dir:", log_dir)


    for epoch in range(1, cfg.epochs + 1):
        model.train()
        total_loss = 0.0

        for step, (u_x, pos_i, neg_i) in enumerate(train_loader):
            if step % 50 == 0:
                print(f"Epoch {epoch} step {step} ...")

            u_x  = u_x.to(device, non_blocking=True).float()
            pos_i = pos_i.to(device, non_blocking=True).float()
            neg_i = neg_i.to(device, non_blocking=True).float()


            pos_s = model(u_x, pos_i)
            neg_s = model(u_x, neg_i)
            loss = bpr_loss(pos_s, neg_s)
            global_step = (epoch - 1) * len(train_loader) + step
            if step % 50 == 0:
                writer.add_scalar("loss/train_step", loss.item(), global_step)


            opt.zero_grad()
            loss.backward()
            opt.step()

            total_loss += loss.item() * u_x.size(0)

        # eval AUC
        model.eval()
        all_y, all_p = [], []
        with torch.no_grad():
            for xb_user, xb_item, yb in test_loader:
                xb_user = xb_user.to(device).float()
                xb_item = xb_item.to(device).float()
                yb = yb.to(device)
                logits = model(xb_user, xb_item)
                prob = torch.sigmoid(logits).cpu().numpy()
                all_p.append(prob)
                all_y.append(yb.cpu().numpy())
        y_true = np.concatenate(all_y)
        y_prob = np.concatenate(all_p)
        auc = roc_auc_score(y_true, y_prob) if len(np.unique(y_true)) > 1 else float("nan")

        avg_loss = total_loss / len(train_loader.dataset)
        print(f"Epoch {epoch:02d} | train_loss={avg_loss:.4f} | test_AUC={auc:.4f}")
        writer.add_scalar("loss/train", avg_loss, epoch)
        writer.add_scalar("metrics/test_auc", auc, epoch)

    writer.flush()
    writer.close()

    # ---- export: model + scaler + metadata ----
    torch.save(model.state_dict(), os.path.join(cfg.artifacts_dir, "twotower_model.pth"))
    joblib.dump(scaler, os.path.join(cfg.artifacts_dir, "scaler.pkl"))
    save_metadata(cfg.artifacts_dir, feature_cols)
    print("Saved model/scaler/feature_cols to", cfg.artifacts_dir)

    # ---- export: ALL item embeddings (unique restaurants) ----
    item_ids, all_item_X = transform_all_items(items, feature_cols, scaler)
    all_item_tensor = torch.from_numpy(all_item_X).to(device)

    model.eval()
    with torch.no_grad():
        item_emb = model.item_tower(all_item_tensor).cpu().numpy()

    np.save(os.path.join(cfg.artifacts_dir, "item_ids.npy"), item_ids)
    np.save(os.path.join(cfg.artifacts_dir, "item_emb.npy"), item_emb)
    print("Saved item_ids.npy and item_emb.npy")

if __name__ == "__main__":
    main()

[DEBUG] feature cols: 54
cutoff: 2019-02-11 02:51:09.200000
train rows: 549146 test rows: 137287
device: cuda
False False
device: cuda
train samples: 404228 test samples: 101057
num features: 54
TensorBoard log_dir: runs\twotower_20260215-223041
Epoch 1 step 0 ...
Epoch 1 step 50 ...
Epoch 1 step 100 ...
Epoch 1 step 150 ...
Epoch 01 | train_loss=0.2287 | test_AUC=0.7938
Epoch 2 step 0 ...
Epoch 2 step 50 ...
Epoch 2 step 100 ...
Epoch 2 step 150 ...
Epoch 02 | train_loss=0.1615 | test_AUC=0.8080
Epoch 3 step 0 ...
Epoch 3 step 50 ...
Epoch 3 step 100 ...
Epoch 3 step 150 ...
Epoch 03 | train_loss=0.1455 | test_AUC=0.8139
Epoch 4 step 0 ...
Epoch 4 step 50 ...
Epoch 4 step 100 ...
Epoch 4 step 150 ...
Epoch 04 | train_loss=0.1333 | test_AUC=0.8218
Epoch 5 step 0 ...
Epoch 5 step 50 ...
Epoch 5 step 100 ...
Epoch 5 step 150 ...
Epoch 05 | train_loss=0.1295 | test_AUC=0.8255
Epoch 6 step 0 ...
Epoch 6 step 50 ...
Epoch 6 step 100 ...
Epoch 6 step 150 ...
Epoch 06 | train_loss=0.1234 | te

In [46]:
import os
import json
import joblib
import numpy as np
import pandas as pd
import torch



def main():
    cfg = Config()
    artifacts = cfg.artifacts_dir

    # load metadata
    feature_cols = json.load(open(os.path.join(artifacts, "feature_cols.json")))
    scaler = joblib.load(os.path.join(artifacts, "scaler.pkl"))

    # load item embeddings
    item_ids = np.load(os.path.join(artifacts, "item_ids.npy"), allow_pickle=True)
    item_emb = np.load(os.path.join(artifacts, "item_emb.npy"))

    # load model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = TwoTower(input_dim=len(feature_cols), hidden_dim=cfg.hidden_dim, embed_dim=cfg.embed_dim).to(device)
    model.load_state_dict(torch.load(os.path.join(artifacts, "twotower_model.pth"), map_location=device))
    model.eval()

    # build user_profiles again (or you can save them too)
    interactions, items = load_raw(cfg.interactions_path, cfg.items_path)
    df = make_merged_df(interactions, items)
    df = make_label(df, cfg.pos_threshold)
    feature_cols = [c for c in feature_cols if c in df.columns]
    user_profiles = build_user_profiles(df, feature_cols)

    # pick a user
    user_id = user_profiles.index[0]  # 你也可以改成你指定的 user_id
    u = user_profiles.loc[user_id, feature_cols].to_numpy(dtype=np.float32).reshape(1, -1)

    # same preprocess: numeric -> fill -> clip -> log1p -> scaler
    u_df = pd.DataFrame(u, columns=feature_cols)
    u_x = numeric_log1p_clip(u_df, feature_cols)
    u_x = scaler.transform(u_x).astype(np.float32)

    user_tensor = torch.from_numpy(u_x).to(device)
    item_emb_t = torch.from_numpy(item_emb).to(device)

    with torch.no_grad():
        user_vec = model.user_tower(user_tensor)              # [1, d]
        scores = torch.matmul(item_emb_t, user_vec.T).squeeze(1)  # [num_items]
        topk = torch.topk(scores, k=cfg.k)

    rec_ids = item_ids[topk.indices.cpu().numpy()].tolist()

    # 去重（保险）
    rec_unique = []
    for bid in rec_ids:
        if bid not in rec_unique:
            rec_unique.append(bid)
    rec_unique = rec_unique[:cfg.k]

    print("user_id:", user_id)
    print("Top-K business_id:")
    print(rec_unique)

if __name__ == "__main__":
    main()

user_id: ---r61b7EpVPkb4UVme5tA
Top-K business_id:
['qsm7SkX60JsajT7Yz248FA', 'Zxi0AGG2Dh-SeqHEMtuUYg', 'Vvd12n0sYII8rUgPhq-XNA', 'j2m_DDgHRAjB0Vx3j8wepw', 'P3vUpcO1EPPmXOZY-Wjdrw', 'e4InIycH2PAJWccBBj0tAA', 'RxrgGdjVD5fFY7DnJoj3tg', 'oB2B1KlyhlY5LTy18jRqzw', 'QWqKTWQ2OiDgo3dzNkpung', 'vA5MHe9LqNQcp8k_CaFkHg']


In [47]:
import os
import json
import joblib
import numpy as np
import pandas as pd
import torch
from math import log2


def ndcg_at_k(hit_positions, k):
    dcg = 0.0
    for r in hit_positions:
        if r <= k:
            dcg += 1.0 / log2(r + 1)
    ideal_hits = min(len(hit_positions), k)
    idcg = sum(1.0 / log2(i + 2) for i in range(ideal_hits))
    return dcg / idcg if idcg > 0 else 0.0


def main():
    cfg = Config()
    artifacts = cfg.artifacts_dir

    # ===== load artifacts =====
    feature_cols = json.load(open(os.path.join(artifacts, "feature_cols.json")))
    scaler = joblib.load(os.path.join(artifacts, "scaler.pkl"))
    item_ids = np.load(os.path.join(artifacts, "item_ids.npy"), allow_pickle=True)
    item_emb = np.load(os.path.join(artifacts, "item_emb.npy"))

    # ===== device (cuda > mps > cpu) =====
    if torch.cuda.is_available():
        device = torch.device("cuda")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")

    # ===== load model =====
    model = TwoTower(input_dim=len(feature_cols), hidden_dim=cfg.hidden_dim, embed_dim=cfg.embed_dim).to(device)
    model.load_state_dict(torch.load(os.path.join(artifacts, "twotower_model.pth"), map_location=device))
    model.eval()

    # ===== rebuild df =====
    interactions, items = load_raw(cfg.interactions_path, cfg.items_path)
    df = make_merged_df(interactions, items)
    df = make_label(df, cfg.pos_threshold)

    # ===== rebuild time split (你要的这一段) =====
    df["review_date"] = pd.to_datetime(df["review_date"], errors="coerce")
    df = df.dropna(subset=["review_date"]).copy()

    cutoff = df["review_date"].quantile(0.8)
    df_train = df[df["review_date"] <= cutoff].copy()
    df_test  = df[df["review_date"] >  cutoff].copy()

    # ===== align feature cols =====
    feature_cols = [c for c in feature_cols if c in df.columns]

    # ===== user_profiles from TRAIN only =====
    user_profiles = build_user_profiles(df_train, feature_cols)

    # ===== seen from TRAIN only =====
    seen = (
        df_train.groupby("user_id")["business_id"]
        .apply(set)
        .to_dict()
    )

    # ===== ground truth from TEST positives only =====
    pos_test = df_test[df_test["label"] == 1][["user_id", "business_id"]].copy()
    gt = pos_test.groupby("user_id")["business_id"].apply(set).to_dict()

    # ===== item embedding tensor =====
    item_emb_t = torch.from_numpy(item_emb).to(device)

    recalls, ndcgs = [], []

    users = list(user_profiles.index)
    users = [u for u in gt.keys() if u in user_profiles.index]


    with torch.no_grad():
        for uid in users:
            true_items = gt.get(uid, set()) - seen.get(uid, set())
            if not true_items:
                continue

            # user vector input (same preprocess as training)
            u = user_profiles.loc[uid, feature_cols].to_numpy(dtype=np.float32).reshape(1, -1)
            u_df = pd.DataFrame(u, columns=feature_cols)
            u_x = numeric_log1p_clip(u_df, feature_cols)
            u_x = scaler.transform(u_x).astype(np.float32)
            user_tensor = torch.from_numpy(u_x).to(device)

            user_vec = model.user_tower(user_tensor)              # [1, d]
            scores = torch.matmul(item_emb_t, user_vec.T).squeeze(1)  # [num_items]

            # ===== mask seen BEFORE topk =====
            seen_set = seen.get(uid, set())
            if seen_set:
                mask = np.isin(item_ids, np.array(list(seen_set), dtype=object))
                if mask.any():
                    scores = scores.masked_fill(torch.from_numpy(mask).to(device), float("-inf"))

            topk = torch.topk(scores, k=cfg.k)
            rec = item_ids[topk.indices.detach().cpu().numpy()].tolist()

            hits = [i for i, bid in enumerate(rec, start=1) if bid in true_items]
            denom = min(len(true_items), cfg.k)
            recall = (len(hits) / denom) if denom > 0 else 0.0

            recalls.append(recall)
            ndcgs.append(ndcg_at_k(hits, cfg.k))

    print(f"cutoff: {cutoff}")
    print(f"Users evaluated: {len(recalls)}")
    print(f"Recall@{cfg.k}: {float(np.mean(recalls)):.4f}")
    print(f"NDCG@{cfg.k}:  {float(np.mean(ndcgs)):.4f}")


if __name__ == "__main__":
    main()

cutoff: 2019-02-11 02:51:09.200000
Users evaluated: 11269
Recall@10: 0.0163
NDCG@10:  0.0234
